In [1]:
!pip install nflows

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 45 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 755.5 MB 26 kB/s s eta 0:00:01    |██▌                             | 57.9 MB 22.3 MB/s eta 0:00:32     |██▌                             | 58.6 MB 22.3 MB/s eta 0:00:32     |██████▌                         | 154.2 MB 5.0 MB/s eta 0:02:01     |█████████████▌                  | 318.4 MB 6.0 MB/s eta 0:01:13     |█████████████████▍              | 411.3 MB 13.6 MB/s eta 0:00:26     |███████████████████████████████▎| 738.9 MB 9.3 MB/s eta 0:00:02
     |████████████████████████████████| 23.7 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 166.0 MB 110 kB/s eta 0:00:014   |█▋                              | 8.2 MB 16.7 MB/s eta 0:00:10     |███████████████████▋            | 102.0 MB 9.9 MB/s eta 0:00:07
     |████████████████████████████████| 410.6 MB 8.8 kB/s  eta 0:00:01     |████████████████▌